# 文本相似度实例

## Step1 导入相关包

In [1]:
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
)
from datasets import load_dataset

/home/pika/App/miniconda3/envs/transformers/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Step2 加载数据集

In [2]:
dataset = load_dataset("json", data_files="./train_pair_1w.json", split="train")
dataset

Dataset({
    features: ['sentence1', 'sentence2', 'label'],
    num_rows: 10000
})

In [3]:
dataset[0]

{'sentence1': '找一部小时候的动画片', 'sentence2': '求一部小时候的动画片。谢了', 'label': '1'}

## Step3 划分数据集

In [4]:
datasets = dataset.train_test_split(test_size=0.2)
datasets

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label'],
        num_rows: 8000
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label'],
        num_rows: 2000
    })
})

## Step4 数据集预处理

In [5]:
import torch

tokenizer = AutoTokenizer.from_pretrained("hfl/chinese-macbert-base")


def process_function(examples):
    sentences = []
    labels = []
    for sen1, sen2, label in zip(
        examples["sentence1"], examples["sentence2"], examples["label"]
    ):
        sentences.append(sen1)
        sentences.append(sen2)
        # -1/1适应cosine loss
        labels.append(1 if int(label) == 1 else -1)
    # input_ids, attention_mask, token_type_ids
    tokenized_examples = tokenizer(
        sentences, max_length=128, truncation=True, padding="max_length"
    )
    tokenized_examples = {
        k: [v[i : i + 2] for i in range(0, len(v), 2)]
        for k, v in tokenized_examples.items()
    }
    tokenized_examples["labels"] = labels
    return tokenized_examples


tokenized_datasets = datasets.map(
    process_function, batched=True, remove_columns=datasets["train"].column_names
)
tokenized_datasets

Map: 100%|██████████| 2000/2000 [00:00<00:00, 10622.65 examples/s]


DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 8000
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 2000
    })
})

In [6]:
print(tokenized_datasets["train"][0])

{'input_ids': [[101, 2442, 3815, 1938, 1062, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [101, 2207, 982, 2207, 3043, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

## Step5 创建模型

$$
\text{loss}(x, y) =
\begin{cases}
1 - \cos(x_1, x_2), & \text{if } y = 1 \\
\max(0, \cos(x_1, x_2) - \text{margin}), & \text{if } y = -1
\end{cases}
$$

In [7]:
from transformers import BertForSequenceClassification, BertPreTrainedModel, BertModel
from typing import Optional
from transformers.configuration_utils import PretrainedConfig
from torch.nn import CosineSimilarity, CosineEmbeddingLoss


class DualModel(BertPreTrainedModel):

    def __init__(self, config: PretrainedConfig, *inputs, **kwargs):
        super().__init__(config, *inputs, **kwargs)
        self.bert = BertModel(config)
        self.post_init()

    def forward(
        self,
        input_ids: Optional[torch.Tensor] = None,
        attention_mask: Optional[torch.Tensor] = None,
        token_type_ids: Optional[torch.Tensor] = None,
        position_ids: Optional[torch.Tensor] = None,
        head_mask: Optional[torch.Tensor] = None,
        inputs_embeds: Optional[torch.Tensor] = None,
        labels: Optional[torch.Tensor] = None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
    ):
        return_dict = (
            return_dict if return_dict is not None else self.config.use_return_dict
        )

        # Step1 分别获取sentenceA 和 sentenceB的输入
        # batch的维度全取，第一个维度取0/1
        senA_input_ids, senB_input_ids = input_ids[:, 0], input_ids[:, 1]
        senA_attention_mask, senB_attention_mask = (
            attention_mask[:, 0],
            attention_mask[:, 1],
        )
        senA_token_type_ids, senB_token_type_ids = (
            token_type_ids[:, 0],
            token_type_ids[:, 1],
        )

        # Step2 分别获取sentenceA 和 sentenceB的向量表示
        senA_outputs = self.bert(
            senA_input_ids,
            attention_mask=senA_attention_mask,
            token_type_ids=senA_token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )

        senA_pooled_output = senA_outputs[1]  # [batch, hidden]

        senB_outputs = self.bert(
            senB_input_ids,
            attention_mask=senB_attention_mask,
            token_type_ids=senB_token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )

        senB_pooled_output = senB_outputs[1]  # [batch, hidden]

        # step3 计算相似度
        # 对应其他模型的logits
        cos = CosineSimilarity()(senA_pooled_output, senB_pooled_output)  # [batch, ]

        # step4 计算loss

        loss = None
        if labels is not None:
            # margin以下的y=-1的样本不算入损失
            loss_fct = CosineEmbeddingLoss(0.3)
            loss = loss_fct(senA_pooled_output, senB_pooled_output, labels)

        output = (cos,)
        return ((loss,) + output) if loss is not None else output


model = DualModel.from_pretrained("hfl/chinese-macbert-base")

## Step6 创建评估函数

In [8]:
import evaluate

acc_metric = evaluate.load("./metric_accuracy.py")
f1_metirc = evaluate.load("./metric_f1.py")

In [9]:
def eval_metric(eval_predict):
    predictions, labels = eval_predict
    ###################################3
    predictions = [int(p > 0.7) for p in predictions]
    labels = [int(l > 0) for l in labels]
    ################################
    # predictions = predictions.argmax(axis=-1)
    acc = acc_metric.compute(predictions=predictions, references=labels)
    f1 = f1_metirc.compute(predictions=predictions, references=labels)
    acc.update(f1)
    return acc

## Step7 创建TrainingArguments

In [10]:
train_args = TrainingArguments(
    output_dir="./dual_model",  # 输出文件夹
    per_device_train_batch_size=32,  # 训练时的batch_size
    per_device_eval_batch_size=32,  # 验证时的batch_size
    logging_steps=10,  # log 打印的频率
    eval_strategy="epoch",  # 评估策略
    save_strategy="epoch",  # 保存策略
    save_total_limit=3,  # 最大保存数
    learning_rate=2e-5,  # 学习率
    weight_decay=0.01,  # weight_decay
    metric_for_best_model="f1",  # 设定评估指标
    load_best_model_at_end=True,
)  # 训练完成后加载最优模型

## Step8 创建Trainer

In [11]:
trainer = Trainer(
    model=model,
    args=train_args,
    tokenizer=tokenizer,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    compute_metrics=eval_metric,
)

## Step9 模型训练

In [12]:
trainer.train()

  1%|▏         | 10/750 [00:05<06:19,  1.95it/s]

{'loss': 0.394, 'grad_norm': 4.751511096954346, 'learning_rate': 1.9733333333333336e-05, 'epoch': 0.04}


  3%|▎         | 20/750 [00:10<06:34,  1.85it/s]

{'loss': 0.2948, 'grad_norm': 6.54201602935791, 'learning_rate': 1.9466666666666668e-05, 'epoch': 0.08}


  4%|▍         | 30/750 [00:16<06:29,  1.85it/s]

...

                                                 
100%|██████████| 750/750 [07:01<00:00,  2.00it/s]

{'eval_loss': 0.17861349880695343, 'eval_accuracy': 0.797, 'eval_f1': 0.7475124378109452, 'eval_runtime': 9.8837, 'eval_samples_per_second': 202.353, 'eval_steps_per_second': 6.374, 'epoch': 3.0}


100%|██████████| 750/750 [07:02<00:00,  1.77it/s]

{'train_runtime': 422.755, 'train_samples_per_second': 56.77, 'train_steps_per_second': 1.774, 'train_loss': 0.17771974213918051, 'epoch': 3.0}


TrainOutput(global_step=750, training_loss=0.17771974213918051, metrics={'train_runtime': 422.755, 'train_samples_per_second': 56.77, 'train_steps_per_second': 1.774, 'total_flos': 3157275967488000.0, 'train_loss': 0.17771974213918051, 'epoch': 3.0})

## Step10 模型评估

In [13]:
trainer.evaluate(tokenized_datasets["test"])

100%|██████████| 63/63 [00:10<00:00,  6.01it/s]


{'eval_loss': 0.17802001535892487,
 'eval_accuracy': 0.7985,
 'eval_f1': 0.7535168195718654,
 'eval_runtime': 10.5864,
 'eval_samples_per_second': 188.923,
 'eval_steps_per_second': 5.951,
 'epoch': 3.0}

## Step11 模型预测

In [14]:
class SentenceSimilarityPipeline:

    def __init__(self, model, tokenizer) -> None:
        self.model = model.bert
        self.tokenizer = tokenizer
        self.device = model.device

    def preprocess(self, senA, senB):
        return self.tokenizer(
            [senA, senB],
            max_length=128,
            # Q：截取的是哪部分？
            # A:senA和senB都有截取
            truncation=True,
            return_tensors="pt",
            padding=True,
        )

    def predict(self, inputs):
        inputs = {k: v.to(self.device) for k, v in inputs.items()}
        # 取bert输出的pooler_output
        # 对应模型定义的step1、step2
        return self.model(**inputs)[1]  # [2, 768]

    def postprocess(self, logits):
        # 对应模型定义的step3
        cos = (
            CosineSimilarity()(logits[None, 0, :], logits[None, 1, :])
            .squeeze()
            .cpu()
            .item()
        )
        return cos

    def __call__(self, senA, senB, return_vector=False):
        inputs = self.preprocess(senA, senB)
        logits = self.predict(inputs)
        print(f"logits.shape: {logits.shape}")
        result = self.postprocess(logits)
        if return_vector:
            return result, logits
        else:
            return result

In [15]:
pipe = SentenceSimilarityPipeline(model, tokenizer)

In [16]:
pipe("我喜欢北京", "明天不行", return_vector=True)

logits.shape: torch.Size([2, 768])


(0.4809381365776062,
 tensor([[-0.9809, -0.6450, -0.6084,  ...,  0.9478, -0.0312, -0.0515],
         [-0.9324,  0.3896, -0.0323,  ..., -0.3455, -0.3624, -0.6469]],
        device='cuda:0', grad_fn=<TanhBackward0>))

In [17]:
pipe("我喜欢北京", "北京是我喜欢的城市", return_vector=True)

logits.shape: torch.Size([2, 768])


(0.7707035541534424,
 tensor([[-0.9809, -0.6450, -0.6084,  ...,  0.9478, -0.0312, -0.0515],
         [-0.9039, -0.2214, -0.1132,  ...,  0.7280, -0.0946,  0.2909]],
        device='cuda:0', grad_fn=<TanhBackward0>))